In [1]:
import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType

model_fp32 = 'assets/landmark_models/lm_model0_opt.onnx'
model_quant = 'assets/landmark_models/lm_model0_opt_quat.onnxx'
calibration_dataset_path = "D:\\data\\LS3D-W.tar\\LS3D-W\\Menpo-3D"
#quantized_model = quantize_dynamic(model_fp32, model_quant, weight_type=QuantType.QUInt8)

ModuleNotFoundError: No module named 'onnx'

In [3]:
import os
import sys
import numpy as np
import re
import abc
import subprocess
import json
import cv2

import onnx
import onnxruntime
from onnx import helper, TensorProto, numpy_helper
from onnxruntime.quantization import quantize_static, calibrate, CalibrationDataReader

##Quat for lm model

class ResNet50DataReader(CalibrationDataReader):
    def __init__(self, calibration_image_folder, augmented_model_path='augmented_model.onnx', size_limit=0):
        self.image_folder = calibration_image_folder
        self.augmented_model_path = augmented_model_path
        self.preprocess_flag = True
        self.enum_data_dicts = []
        self.datasize = 0
        

        self.size_limit = size_limit


    def get_next(self):
        if self.preprocess_flag:
            self.preprocess_flag = False
            session = onnxruntime.InferenceSession(self.augmented_model_path, None)
            (_, height, width, _) = session.get_inputs()[0].shape
            nhwc_data_list = preprocess_func(self.image_folder, 224, 224, size_limit=self.size_limit)
            input_name = session.get_inputs()[0].name
            print(input_name, len(nhwc_data_list))
            self.datasize = len(nhwc_data_list)
            self.enum_data_dicts = iter([{input_name: nhwc_data} for nhwc_data in nhwc_data_list])
        return next(self.enum_data_dicts, None)


def preprocess_func(images_folder, height, width, size_limit=0):
    '''
    Loads a batch of images and preprocess them
    parameter images_folder: path to folder storing images
    parameter height: image height in pixels
    parameter width: image width in pixels
    parameter size_limit: number of images to load. Default is 0 which means all images are picked.
    return: list of matrices characterizing multiple images
    '''
    mean = np.float32(np.array([0.485, 0.456, 0.406]))
    std = np.float32(np.array([0.229, 0.224, 0.225]))
    mean = mean / std
    std = std * 255.0
    mean = - mean
    std = 1.0 / std
    mean_32 = np.tile(mean, [32, 32, 1])
    std_32 = np.tile(std, [32, 32, 1])
    mean_224 = np.tile(mean, [224, 224, 1])
    std_224 = np.tile(std, [224, 224, 1])
        
    image_names = os.listdir(images_folder)
    if size_limit > 0 and len(image_names) >= size_limit:
        batch_filenames = [image_names[i] for i in range(size_limit)]
    else:
        batch_filenames = image_names
    unconcatenated_batch_data = []
    print(f"Collect {len(batch_filenames)} files")
    for image_name in batch_filenames:
        if image_name.endswith(".jpg"):
            image_filepath = images_folder + '/' + image_name
            frame = cv2.imread(image_filepath, cv2.IMREAD_COLOR)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            im = cv2.resize(frame, (width, height), interpolation=cv2.INTER_LINEAR)[:,:,::-1]
            im = im * std_224 + mean_224
            im = np.expand_dims(im, 0)
            im = np.transpose(im, (0,3,1,2))
            unconcatenated_batch_data.append(im)
    batch_data = np.concatenate(np.expand_dims(unconcatenated_batch_data, axis=0), axis=0)
    return batch_data


dr = ResNet50DataReader(calibration_dataset_path, model_fp32, 10000)
quantize_static(model_fp32, model_quant, dr)
print('Calibrated and quantized model saved.')



Collect 10000 files
input 5000
Calibrated,quantized parameters calculated and returned.
Calibrated and quantized model saved.
